In [ ]:
!pip install --upgrade tensorflow

In [ ]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices())


2.15.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Import Dependencies

In [ ]:
import keras
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils import class_weight
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta, RMSprop
from keras.models import Sequential, model_from_json
from keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPool2D,MaxPooling2D,AveragePooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
import os
from glob import glob
#import matplotlib
#import matplotlib.pyplot as plt
import random
import cv2
import pandas as pd
import numpy as np
#import matplotlib.gridspec as gridspec
#import seaborn as sns
import zlib
import itertools
import sklearn
import itertools
import scipy
import skimage
from skimage.transform import resize
import csv
from tqdm import tqdm
from sklearn import model_selection
from sklearn.model_selection import train_test_split, learning_curve,KFold,cross_val_score,StratifiedKFold
from sklearn.utils import class_weight, shuffle
from sklearn.metrics import confusion_matrix
#from imblearn.over_sampling import RandomOverSampler
#from imblearn.under_sampling import RandomUnderSampler
from keras.applications.mobilenet import MobileNet
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from tensorflow.keras.callbacks import ModelCheckpoint
import psutil
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import warnings
warnings.filterwarnings("ignore")
#%matplotlib inline

Load Data

In [ ]:
class MetricsCheckpoint(Callback):
  """Callback that saves metrics after each epoch"""
  def __init__(self, savepath):
    super(MetricsCheckpoint, self).__init__()
    self.savepath = savepath
    self.history = {}
  def on_epoch_end(self, epoch, logs=None):
    for k, v in logs.items():
        self.history.setdefault(k, []).append(v)
    np.save(self.savepath, self.history)

In [ ]:
imageSize=50
train_dir = "input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/"
test_dir =  "input/asl-alphabet/asl_alphabet_test/asl_alphabet_test/"
from tqdm import tqdm
def get_data(folder):
  """
  Load the data and labels from the given folder.
  """
  X = []
  y = []
  for folderName in os.listdir(folder):
    if not folderName.startswith('.'):
      if 'A' in folderName:
        label = 0
      elif 'B' in folderName:
        label = 1
      elif 'C' in folderName:
        label = 2
      elif 'D' in folderName:
        label = 3
      elif 'E' in folderName:
        label = 4
      elif 'F' in folderName:
        label = 5
      elif 'G' in folderName:
        label = 6
      elif 'H' in folderName:
        label = 7
      elif 'I' in folderName:
        label = 8
      elif 'J' in folderName:
        label = 9
      elif 'K' in folderName:
        label = 10
      elif 'L' in folderName:
        label = 11
      elif 'M' in folderName:
        label = 12
      elif 'N' in folderName:
        label = 13
      elif 'O' in folderName:
        label = 14
      elif 'P' in folderName:
        label = 15
      elif 'Q' in folderName:
        label = 16
      elif 'R' in folderName:
        label = 17
      elif 'S' in folderName:
        label = 18
      elif 'T' in folderName:
        label = 19
      elif 'U' in folderName:
        label = 20
      elif 'V' in folderName:
        label = 21
      elif 'W' in folderName:
        label = 22
      elif 'X' in folderName:
        label = 23
      elif 'Y' in folderName:
        label = 24
      elif 'Z' in folderName:
        label = 25
      elif 'del' in folderName:
        label = 26
      elif 'nothing' in folderName:
        label = 27
      elif 'space' in folderName:
        label = 28
      else:
        label = 29
      try:
        for image_filename in tqdm(os.listdir(folder + folderName)):
          img_file = cv2.imread(folder + folderName + '/' + image_filename)
          if img_file is not None:
            # TODO: Convert the image to Nodes before loading
            img_file = skimage.transform.resize(img_file, (imageSize, imageSize, 3))
            img_arr = np.asarray(img_file)
            X.append(img_arr)
            y.append(label)
      except:
        img_file = cv2.imread(folder + folderName)
        if img_file is not None:
          img_file = skimage.transform.resize(img_file, (imageSize, imageSize, 3))
          img_arr = np.asarray(img_file)
          X.append(img_arr)
          y.append(label)

  X = np.asarray(X)
  y = np.asarray(y)
  return X,y

X_test, y_test= get_data(test_dir) # Too few images
X_train, y_train = get_data(train_dir)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

# Encode labels to hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
y_trainHot = to_categorical(y_train, num_classes = 29)
y_testHot = to_categorical(y_test, num_classes = 29)

In [ ]:
# Shuffle data to permit further subsampling
#from tensorflow_metrics.callbacks import MetricsCheckpoint
X_train, y_trainHot = shuffle(X_train, y_trainHot, random_state=13)
X_test, y_testHot = shuffle(X_test, y_testHot, random_state=13)
X_train = X_train[:30000]
X_test = X_test[:30000]
y_trainHot = y_trainHot[:30000]
y_testHot = y_testHot[:30000]

Train Model

In [ ]:
map_characters1 = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: 'del', 27: 'nothing', 28: 'space'}
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
# Create a dictionary of class weights
class_weight1 = {class_idx: weight for class_idx, weight in zip(np.unique(y_train), class_weights)}
weight_path1 = 'input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
weight_path2 = 'input/keras-pretrained-models/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pretrained_model_1 = VGG16(weights = weight_path1, include_top=False, input_shape=(imageSize, imageSize, 3))
#pretrained_model_2 = InceptionV3(weights = weight_path2, include_top=False, input_shape=(imageSize, imageSize, 3))

In [ ]:
from tensorflow.keras.optimizers import legacy

# Replace your optimizer definition with the legacy version
#optimizer1 = legacy.Adam(lr=0.0001)

optimizer1 = keras.optimizers.Adam()
optimizer2 = keras.optimizers.RMSprop(learning_rate=0.0001)

In [ ]:
def pretrainedNetwork(xtrain,ytrain,xtest,ytest,pretrainedmodel,pretrainedweights,classweight,numclasses,numepochs,optimizer,labels):
  base_model = pretrained_model_1 # Topless
  # Add top layer
  x = base_model.output
  x = Flatten()(x)
  predictions = Dense(numclasses, activation='softmax')(x)
  model = Model(inputs=base_model.input, outputs=predictions)
  # Train top layer
  for layer in base_model.layers:
    layer.trainable = False
  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  callbacks_list = [keras.callbacks.EarlyStopping(monitor='validation_accuracy', patience=3, verbose=1)]
  model.summary()
  # Fit model
  #optimizer1.build(model.trainable_variables)
  history = model.fit(xtrain,ytrain, epochs=numepochs, class_weight=classweight, validation_data=(xtest,ytest), verbose=1,callbacks = [MetricsCheckpoint('logs')])
  # Evaluate model
  score = model.evaluate(xtest,ytest, verbose=0)
  print('\nKeras CNN - accuracy:', score[1], '\n')
  y_pred = model.predict(xtest)
  print('\n', sklearn.metrics.classification_report(np.where(ytest > 0)[1], np.argmax(y_pred, axis=1), target_names=list(labels.values())), sep='')
  Y_pred_classes = np.argmax(y_pred,axis = 1)
  Y_true = np.argmax(ytest,axis = 1)
  confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
  return model


In [ ]:
trained_model = pretrainedNetwork(X_train, y_trainHot, X_test, y_testHot,pretrained_model_1,weight_path1,class_weight1,29,10,optimizer1,map_characters1)

In [ ]:
trained_model.save('new_model.h5')

Test an Image

In [ ]:
## TODO: convert each jpg to nodes in the load_data function